# Interpolation

As mentioned in [ffc_tests.ipynb](ffc_tests.ipynb) `interpolate()` cannot work in Hermite function spaces without extending the interface for `ufc::function` to include differentiation. This comes from the fact that the standard (local) nodal interpolant is given by

$$ I_K (f) = \sum_{i = 1}^{n_K} L_i (f) \phi_i $$

for all dofs $L_i$ over cell $K$. In particular we need to let the partial differentiation along each spatial coordinate (in the notation of the paper $L_{\alpha_i}, 1 \leq i \leq d$) act on the function $f$ and use this value as the coefficient for the corresponding shape function $\phi_{\alpha_i}$ in the representation in the global basis of $V_h$.

This notebook will contain progress in this matter.

## The current situation (FEniCS 2017.0.1)

The default implementation (in `ffc/evaluatedof.py:_generate_body()`) of code generation for

```c++
double evaluate_dof(std::size_t i,
                      const ufc::function& f,
                      const double * coordinate_dofs,
                      int cell_orientation,
                      const ufc::cell& c) const final override
```

does the following:

1. Get coordinate for dof `i`.
2. Call `f.evaluate()` on this coordinate.

I.e. it assumes that all dofs are point evaluations!

## A hack for `interpolate()`

In [ ]:
from dolfin import *
%matplotlib inline
import matplotlib.pyplot as pl
import autograd.numpy as np

This won't work:

In [ ]:
V = FunctionSpace(UnitSquareMesh(10, 10), 'Hermite', 3)
f = Expression("x[0]*x[1]", degree=3)
#v = interpolate(f, V)
#_ = plot(v, cmap='bone')

We need to be able to take derivatives. A first attempt might be to [use UFL](#Using-UFL) to compute derivatives of `Expressions` but I can't get this to work for some reason...

Instead, we use actual python functions and compute their derivatives with `autograd` (see [utils.py](utils.py)).

In [ ]:
from utils import make_derivatives
@make_derivatives
def fun2d(x, y):
    return x*y**2+2*x

In [ ]:
tt = np.linspace(0, 1, 10)
xx, yy = np.meshgrid(tt, tt)
pp = np.dstack((xx,yy))

points_shape = pp.shape[:-1]
zz = np.zeros(points_shape)
for i,j in np.ndindex(points_shape):
    zz[i,j] = f.evaluate(pp[i,j], {f: fun2d}, (), None, derivatives=(0,1))

In [ ]:
_ = pl.imshow(zz, aspect='auto', cmap='bone')

In [ ]:
from FIAT import functional
from ffc import fiatinterface

def interpolate_hermite(f, V):
    """ Nodal interpolation for elements  .
    
    If V is built with any elements other than hermite, use
    dolfin.interpolate(). Otherwise:
    
    NOTE: This is just a hack and requires an actual python function
          which we can evaluate and differentiate.
    
    Arguments:
    ----------
        f: Callable taking exactly one np.ndarray of size equal to the
           topological dimension of the mesh.
           Use the decorator @make_derivatives to use autograd's AD.
           Example:

                @make_derivatives
                def fun2d(x, y):
                    return x*y
            
            Now fun2d() has signature fun2d(np.ndarray, derivatives=())

        V: FunctionSpace using Hermite elements.

    Returns:
    --------
        A Function in V.
    """
    print("WARNING: Nodal interpolation for Hermite dofs. HACK HACK.")
    tdim = V.element().topological_dimension()
    e = fiatinterface.create_element(V.ufl_element())
    
    # This mask filters out the Hermite dofs from the list of dofs of a cell:
    hmask = np.array(map(lambda f: isinstance(f, functional.PointDerivative),
                        e.dual_basis()))
    lmask = np.array(map(lambda f: isinstance(f, functional.PointEvaluation),
                        e.dual_basis()))
    assert ((hmask | lmask) == True).size == len(e.dual_basis()),\
           "Only Lagrange and Hermite dofs supported."

    #print("hmask = %s" % hmask)
    uv = np.zeros(V.dim())
    dm = V.dofmap()
    dof_coordinates = V.tabulate_dof_coordinates().reshape((-1, tdim))
    if tdim == 1:
        derivatives = [(1,)]*2
    elif tdim == 2:
        derivatives = [(1,0),(0,1)]*3
    elif tdim == 3:
        derivatives = [(1,0,0),(0,1,0),(0,0,1)]*4
    for i in range(V.mesh().num_cells()):
        dofs = dm.cell_dofs(i)
        hermite_dofs = dofs[hmask]
        lagrange_dofs = dofs[lmask]
        for dd, hdof in zip(derivatives, hermite_dofs):
            uv[hdof] = f(dof_coordinates[hdof], derivatives=dd)
            #print("hdof=%d, coords=%s, derivs=%s" % (hdof, dof_coordinates[hdof], dd))
        for ldof in lagrange_dofs:
            uv[ldof] = f(dof_coordinates[ldof], derivatives=())
    u = Function(V)
    u.vector().set_local(uv)

    return u

In [ ]:
V = FunctionSpace(UnitSquareMesh(20, 20), 'DKT', 3)
u = interpolate_hermite(fun2d, V)
v = project(Expression("x[0]*x[1]*x[1]+2*x[0]", degree=3), V)
w = Function(V, u.vector() - v.vector())

In [ ]:
pl.figure(figsize=(12,4))
pl.subplot(1,3,1)
plot(u, title="Interpolation into H3", cmap='bone')
pl.subplot(1,3,2)
plot(v, title="Projection onto H3", cmap='bone')
pl.subplot(1,3,3)
_ = plot(w, title="Difference", cmap='bone')

In [ ]:
from IPython.display import Math
Math("$||u_h - u_p||_{L^2} = %.4f, \  ||u_h - u_p||_{L^{\infty}} = %.4f$" % 
     (norm(w), norm(w.vector(), norm_type='linf')))

In [ ]:
u_vert = u.compute_vertex_values()
v_vert = v.compute_vertex_values()
print("%.2f%% of vertices where u and v disagree more than 1e-6" % 
      (100.*np.nonzero(np.abs(u_vert-v_vert) > 1e-6)[0].size / V.dim()))

# Constants

In [ ]:
def make_constant(value, V):
    """ Returns a constant Function over V.
    This only makes sense when V is built with elements using
    Hermite dofs
    
    Arguments
    ---------
        value: a scalar.
        V: FunctionSpace of elements using Hermite and Lagrange dofs.
    Returns
    -------
        A Function in V.
    """
    
    def fun(x, derivatives=()):
        return value if sum(derivatives) == 0 else 0.

    return interpolate_hermite(fun, V)

## Testing the norms

Something is quite off...

In [ ]:
@make_derivatives
def ftest(x, y):
    return x**2+y**2

V = FunctionSpace(UnitSquareMesh(20, 20), 'DKT', 3)
w1 = interpolate_hermite(ftest, V)
w2 = project(Expression("x[0]*x[0]+x[1]*x[1]", degree=3), V)
print("           w1\t\tw2\t\texact " % ())
print("L2       : %f, %f, %f" % (norm(w1, 'L2'), norm(w2, 'L2'), np.sqrt(28/45.)))
print("H10 norms: %f, %f, %f" % (norm(w1, 'H10'), norm(w2, 'H10'), np.sqrt(8/3.)))
print("H1 norms : %f, %f, %f" % (norm(w1, 'H1'), norm(w2, 'H1'), np.sqrt(8/3. + 28/45.)))
#Math("\\text{Exact: }||f||_{L^2} = %f,\ ||f||_{H^1_0} = %f,\ ||f||_{H^1} = %f" %
#      (np.sqrt(28/45.), np.sqrt(8/3.), np.sqrt(8/3. + 28/45.)))

In [ ]:
dw1 = grad(w1)
ddw1 = inner(dw1, dw1)

## Using UFL

A couple of tests trying to use AD on expressions in order to implement an `interpolate()` 

In [ ]:
from ufl.algorithms.apply_derivatives import apply_derivatives

V = FunctionSpace(UnitIntervalMesh(2), 'Hermite', 3)
x = SpatialCoordinate(V.mesh())
f = x[0]**2
str(f)
#replace(f, {x:0.})  # Doesn't work
dfdx = apply_derivatives(diff(f, x))
#replace(dfdx, {x: 0.})  # Doesn't work

## A test

The first thing to check once `evaluate_dof()` has been implemented for Hermite elements is a constant:

In [ ]:
u = interpolate(Constant(2.0), V)
_ = plot(u, cmap='bone')

# Interface

In [ ]:
def __nbinit__():
    global __all__
    __all__ = ['interpolate_hermite', 'make_constant']